In [ ]:
import numpy as np
import scipy.io
from midiutil import MIDIFile
from utils import *

In [ ]:
key_time = 50e-3*0.5
Fs = 48000

#keys = scipy.io.loadmat('MATLAB_data/31Jan_jc_upper_octave_noise.mat')['key']
keys = scipy.io.loadmat('MATLAB_data/31Jan_jc_key.mat')['key']
notes_list = scipy.io.loadmat('notes_100.mat')['notes']
freq_list = scipy.io.loadmat('freq_480_octave_down.mat')['f_cropped']
# notes_list = scipy.io.loadmat('notes_88.mat')['notes']
# freq_list = scipy.io.loadmat('freq_480.mat')['f_cropped']
#freq_list = scipy.io.loadmat('freq_441.mat')['f_cropped']
notes_list = notes_list.reshape(np.size(notes_list))
freq_list = freq_list.reshape(np.size(freq_list))

note_num = 100 #88+12, 12 from upper octave

In [ ]:
intensity_map = scipy.io.loadmat('MATLAB_data/31Jan_jc_int.mat')['filtered_int_db']

In [ ]:
def intensity2sign_new(intensity):
    if intensity < 35:
        return 'p'
    elif intensity < 55:
        return 'mp'
    elif intensity < 75:
        return 'mf'
    else:
        return 'f'

In [ ]:
def get_intensity(cur_key, cur_freq, notes_list):
    wave_freq = notes_list[cur_key-1]
    if cur_key == 1:
        sigma = notes_list[1]-notes_list[0]
    else:
        sigma = notes_list[cur_key-1]-notes_list[cur_key-2]
    sigma = sigma/2
    intensity = np.exp(-0.5 *((cur_freq-wave_freq)/sigma)**2)
    #print('cur', cur_freq,'wave',wave_freq)
    return intensity

In [ ]:
def picknotes(cur_key, cur_freq, cur_int, notes, notes_list):
    j = 0
    while j < len(cur_key):
        if cur_key[j] == 0:
            j += 1
            continue
        if cur_key[j] > 88:
            pitch = cur_key[j] - 12
        else:
            pitch = cur_key[j]
        start = j
        ori_int = cur_int[j]
        while cur_key[j] != 0 and j < len(cur_key)-1:
            j += 1
            ori_int = max(ori_int,cur_int[j])
        
        #intensity = get_intensity(cur_key[j-1],cur_freq,notes_list) * ori_int
        intensity = get_intensity(cur_key[j-1],cur_freq,notes_list)
        #intensity = get_intensity(cur_key[j-1],cur_freq,notes_list)
        #intensity_sign = intensity2sign_new(intensity)
        intensity_sign = intensity2sign(intensity)
        notes.append(Note(pitch, start, j - 1, intensity, intensity_sign))
        j += 1
    return notes

In [ ]:
notes = []
for i in range(np.shape(keys)[0]):
    cur_key = keys[i]
    cur_int = intensity_map[i]
    cur_freq = freq_list[i]
    notes = picknotes(cur_key, cur_freq, cur_int, notes, notes_list)

print(len(notes))

In [ ]:
def remove_repetitive(notes,note_num):
    #changed
    note_set = [0]*note_num
    i = 0
    while i < len(notes):
        note = notes[i]
        if note_set[note.pitch-1] == 0:
            note_set[note.pitch-1] = [note]
        else:
            delete = False
            for n in note_set[note.pitch-1]:
                # print(n)
                # print(n.start)
                if n.start == note.start and n.end == note.end:
                    n.intensity = max(n.intensity, note.intensity)
                    #n.intensity_sign = intensity2sign_new(n.intensity)
                    n.intensity_sign = intensity2sign(n.intensity)
                    del notes[i]
                    delete = True
                    i -= 1
                    break
            if not delete:
                note_set[note.pitch-1].append(note)
        i += 1
    return notes

In [ ]:
#notes = sorted(notes, key = lambda note: note.start)
notes = sorted(notes, key = lambda note: (note.pitch, note.start, note.end))

In [ ]:
pprint(notes)

In [ ]:
new_notes = remove_repetitive(notes,note_num)
pprint(new_notes)
print(len(new_notes))

In [ ]:
nnew_notes = multichannel(new_notes,note_num)
for note in nnew_notes:
    print(note)

In [ ]:
import matplotlib.pyplot as plt
def plot_notes(note_list):
    time_lower = []
    pitch_lower = []
    time_upper = []
    pitch_upper = []
    for note in note_list:
        if note.pitch < 89:
            time_lower.append(note.start)
            pitch_lower.append(note.pitch)
        else:
            time_upper.append(note.start)
            pitch_upper.append(note.pitch)
        
    plt.scatter(time_upper,pitch_upper,s=1)
    plt.scatter(time_lower,pitch_lower,s=1)

In [ ]:
def plot_notes(note_list):
    time =[]
    pitch = []
    for note in note_list:
        time.append(note.start)
        pitch.append(note.pitch)
    plt.scatter(time,pitch,s=1)

In [ ]:
plot_notes(new_notes)

In [ ]:
mf = MIDIFile(1)     # only 1 track
track = 0   # the only track
time = 0    # start at the beginning
mf.addTrackName(track, time, "Sample Track")
mf.addTempo(track, time, 1200)

In [ ]:
for note in nnew_notes:
    # v = int(20+note.sign()*20)
    v = int(50+note.sign()*12.5)
    mf.addNote(track,note.channel,note.pitch+20, note.start, note.end-note.start+1,v)

In [ ]:
with open("midifile/31Jan_jc_upper_octave_int_4.mid", 'wb') as outf:
   mf.writeFile(outf)

In [ ]:
# print(len(nnew_notes))
# print(len(notes))

In [ ]:
# def picknotes_old(cur_key,cur_freq):
#     j = 0
#     while j < len(cur_key):
#         if cur_key[j] != 0:
            
#             d = {}
#             d['note'] = cur_key[j]
#             d['start'] = j
#             while cur_key[j] != 0 and j < len(cur_key)-1:
#                 j += 1
#             d['end'] = j - 1
#             intensity = get_intensity(cur_key[j-1],cur_freq,notes_list[cur_key[j-1]-1])
#             d['intensity'] = intensity
#             notes.append(d)
#         j += 1